# Organism

In [4]:
! nvidia-smi

Tue Oct 13 14:10:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    32W / 250W |    363MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPool2D, ReLU, ELU, LeakyReLU, Flatten, Dense, AveragePooling2D
import pprint
pp = pprint.PrettyPrinter(indent=4)

import numpy as np
np.random.seed(666)
tf.random.set_seed(666)

In [2]:
options = {
    'a_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'a_include_BN': [True, False],
    'a_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'activation_type': [ReLU, ELU, LeakyReLU],
    'b_filter_size': [(1,1), (3,3), (5,5), (7,7), (9,9)],
    'b_include_BN': [True, False],
    'b_output_channels': [8, 16, 32, 64, 128, 256, 512],
    'include_pool': [True, False],
    'pool_type': [MaxPool2D, AveragePooling2D]
    }

In [3]:
# Load the training and testing set of CIFAR10
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

X_train = X_train.astype('float32')
X_train = X_train/255.

X_test = X_test.astype('float32')
X_test = X_test/255.

y_train = tf.reshape(tf.one_hot(y_train, 10), shape=(-1, 10))
y_test = tf.reshape(tf.one_hot(y_test, 10), shape=(-1, 10))

# Create TensorFlow dataset
BATCH_SIZE = 1024
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_ds = train_ds.shuffle(1024).cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))
test_ds = test_ds.cache().batch(BATCH_SIZE).prefetch(AUTOTUNE)

170500096/170498071 [==============================] - 4s 0us/step


In [5]:
class Organism:

  def __init__(self, chromosome={}, phase=0):
    self.phase = phase
    self.chromosome = chromosome
    if len(self.chromosome) != 0:
      self.build_model()

  def build_model(self):
    if self.phase == 0:
      keras.backend.clear_session()
      inputs = Input(shape=(32,32,3))
      x = Conv2D(filters=self.chromosome['a_output_channels'],
                 kernel_size=self.chromosome['a_filter_size'],
                 use_bias=self.chromosome['a_include_BN'])(inputs)
      if self.chromosome['a_include_BN']:
        x = BatchNormalization()(x)
      x = self.chromosome['activation_type']()(x)
      if self.chromosome['include_pool']:
        x = self.chromosome['pool_type']()(x)

      x = Conv2D(filters=self.chromosome['b_output_channels'],
                 kernel_size=self.chromosome['b_filter_size'],
                 use_bias=self.chromosome['b_include_BN'])(x)
      if self.chromosome['b_include_BN']:
        x = BatchNormalization()(x)
      x = self.chromosome['activation_type']()(x)

      x = Flatten()(x)
      x = Dense(10, activation='softmax')(x)

      self.model = tf.keras.Model(inputs=[inputs], outputs=[x])
      self.model.compile(optimizer='adam',
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])
    else:
      print('Phase under construction')
      
  def fitnessFunction(self, train_ds, test_ds):
    self.model.fit(train_ds,
                   epochs=5,
                   verbose=1)
    _, self.fitness = self.model.evaluate(test_ds, verbose=0)
  
  def crossover(self, partner):
    child = Organism(chromosome={}, phase=0)
    endpoint = np.random.randint(low=0, high=len(self.chromosome))
    print(endpoint)
    for idx, key in enumerate(self.chromosome):
      if idx <= endpoint:
        child.chromosome[key] = self.chromosome[key]
      else:
        child.chromosome[key] = partner.chromosome[key]
    child.build_model()
    return child
  
  def mutation(self, mutationRate):
    for idx, key in enumerate(self.chromosome):
      if np.random.rand() <= mutationRate:
        self.chromosome[key] = options[key][np.random.randint(len(options[key]))]
  
  def show(self):
    pp.pprint(self.chromosome)

# Generation

In [6]:
def random_hyper():
  return {
    'a_filter_size': options['a_filter_size'][np.random.randint(len(options['a_filter_size']))],
    'a_include_BN': options['a_include_BN'][np.random.randint(len(options['a_include_BN']))],
    'a_output_channels': options['a_output_channels'][np.random.randint(len(options['a_output_channels']))],
    'activation_type': options['activation_type'][np.random.randint(len(options['activation_type']))],
    'b_filter_size': options['b_filter_size'][np.random.randint(len(options['b_filter_size']))],
    'b_include_BN': options['b_include_BN'][np.random.randint(len(options['b_include_BN']))],
    'b_output_channels': options['b_output_channels'][np.random.randint(len(options['b_output_channels']))],
    'include_pool': options['include_pool'][np.random.randint(len(options['include_pool']))],
    'pool_type': options['pool_type'][np.random.randint(len(options['pool_type']))]
    }

def softmax(x):
  e_x = np.exp(x - np.max(x))
  return e_x / e_x.sum()

In [12]:
class Generation:
	
	def __init__(self, mutationRate, population_size):
		self.population_size = population_size
		self.population = []
		self.generation_number = 0
		self.finished = False
		self.mutationRate = mutationRate

		# creating the first population GENERATION 1
		# can be thought of as the setup function
		for idx in range(self.population_size):
			self.population.append(Organism(random_hyper(), phase=0))
		# calculates the fitness of all the organisms

		self.calcFitness()

	def calcFitness(self):
		# This function is used to calculate the fitness of all
		# the individuals of the population.
		for individuals in self.population:
			individuals.fitnessFunction(train_ds, test_ds)
	
	# genrate the children and place them in the previous generation
	# to make new generation
	def generate(self):
		fitness = [ind.fitness for ind in self.population]
		sorted_index = np.argsort(fitness).tolist()

		temp_population = self.population.copy()
		# create children with pairs
		for idx in range (self.population_size):
			if idx < self.population_size//2:
				self.population[idx] = temp_population[sorted_index[self.population_size-idx-1]]
			else:
				parents = np.random.choice(temp_population, size=(2,), replace=False, p=softmax(fitness))
				A=parents[0]
				B=parents[1]
				child=A.crossover(B)
				child.mutation(self.mutationRate)
				child.fitnessFunction(train_ds, test_ds)
				self.population[idx]=child
		self.generation_number+=1

	# evaluates the whole generation's fitness
	def evaluate(self):
		fitness = [ind.fitness for ind in self.population]
		most_fit = max(fitness)
		print('Generation: {}'.format(self.generation_number))
		print('Best fitness: {:0.2f}'.format(most_fit))
		print('Average fitness: {:0.2f}'.format(sum(fitness)/len(fitness)))
		if most_fit > 0.6 :
			self.finished=True
			indices = [index for index, value in enumerate(fitness) if value == most_fit]
			for index in indices:
				self.population[index].show()

In [13]:
population_size = 10
mutationRate = 0.1
pop = Generation(mutationRate, population_size)

while not(pop.finished):
	pop.generate()
	pop.evaluate()

Epoch 1/5
49/49 [==============================] - 1s 18ms/step - loss: 1.8504 - accuracy: 0.3462
Epoch 2/5
49/49 [==============================] - 1s 17ms/step - loss: 1.4596 - accuracy: 0.4805
Epoch 3/5
49/49 [==============================] - 1s 17ms/step - loss: 1.3226 - accuracy: 0.5293
Epoch 4/5
49/49 [==============================] - 1s 16ms/step - loss: 1.2457 - accuracy: 0.5577
Epoch 5/5
49/49 [==============================] - 1s 16ms/step - loss: 1.1943 - accuracy: 0.5769
Epoch 1/5
49/49 [==============================] - 1s 13ms/step - loss: 1.8822 - accuracy: 0.3270
Epoch 2/5
49/49 [==============================] - 1s 12ms/step - loss: 1.5492 - accuracy: 0.4411
Epoch 3/5
49/49 [==============================] - 1s 12ms/step - loss: 1.4728 - accuracy: 0.4699
Epoch 4/5
49/49 [==============================] - 1s 13ms/step - loss: 1.4241 - accuracy: 0.4876
Epoch 5/5
49/49 [==============================] - 1s 13ms/step - loss: 1.3865 - accuracy: 0.5018
Epoch 1/5
49/49 [===

49/49 [==============================] - 9s 178ms/step - loss: 2.3147 - accuracy: 0.2452
Epoch 2/5
49/49 [==============================] - 8s 160ms/step - loss: 1.6394 - accuracy: 0.4153
Epoch 3/5
49/49 [==============================] - 8s 160ms/step - loss: 1.5061 - accuracy: 0.4696
Epoch 4/5
49/49 [==============================] - 8s 160ms/step - loss: 1.4204 - accuracy: 0.5023
Epoch 5/5
49/49 [==============================] - 8s 160ms/step - loss: 1.3589 - accuracy: 0.5252
Epoch 1/5
49/49 [==============================] - 1s 15ms/step - loss: 1.9421 - accuracy: 0.3175
Epoch 2/5
49/49 [==============================] - 1s 15ms/step - loss: 1.5783 - accuracy: 0.4489
Epoch 3/5
49/49 [==============================] - 1s 15ms/step - loss: 1.4431 - accuracy: 0.4960
Epoch 4/5
49/49 [==============================] - 1s 14ms/step - loss: 1.3631 - accuracy: 0.5227
Epoch 5/5
49/49 [==============================] - 1s 14ms/step - loss: 1.3158 - accuracy: 0.5389
Epoch 1/5
49/49 [========

49/49 [==============================] - 6s 132ms/step - loss: 2.0459 - accuracy: 0.2859
Epoch 2/5
49/49 [==============================] - 6s 118ms/step - loss: 1.5481 - accuracy: 0.4503
Epoch 3/5
49/49 [==============================] - 6s 118ms/step - loss: 1.3834 - accuracy: 0.5157
Epoch 4/5
49/49 [==============================] - 6s 118ms/step - loss: 1.2840 - accuracy: 0.5549
Epoch 5/5
49/49 [==============================] - 6s 118ms/step - loss: 1.2157 - accuracy: 0.5790
6
Epoch 1/5
49/49 [==============================] - 1s 14ms/step - loss: 1.8709 - accuracy: 0.3370
Epoch 2/5
49/49 [==============================] - 1s 14ms/step - loss: 1.5167 - accuracy: 0.4677
Epoch 3/5
49/49 [==============================] - 1s 14ms/step - loss: 1.3950 - accuracy: 0.5126
Epoch 4/5
49/49 [==============================] - 1s 14ms/step - loss: 1.3334 - accuracy: 0.5343
Epoch 5/5
49/49 [==============================] - 1s 14ms/step - loss: 1.2953 - accuracy: 0.5489
1
Epoch 1/5
49/49 [====

49/49 [==============================] - 7s 147ms/step - loss: 3.0678 - accuracy: 0.1952
Epoch 2/5
49/49 [==============================] - 7s 140ms/step - loss: 1.8107 - accuracy: 0.3622
Epoch 3/5
49/49 [==============================] - 7s 142ms/step - loss: 1.6704 - accuracy: 0.4180
Epoch 4/5
49/49 [==============================] - 7s 141ms/step - loss: 1.5718 - accuracy: 0.4511
Epoch 5/5
49/49 [==============================] - 7s 141ms/step - loss: 1.4853 - accuracy: 0.4780
6
Epoch 1/5
 2/49 [>.............................] - ETA: 6s - loss: 3.3577 - accuracy: 0.1162WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0396s vs `on_train_batch_end` time: 0.1212s). Check your callbacks.


49/49 [==============================] - 8s 159ms/step - loss: 2.1412 - accuracy: 0.2975
Epoch 2/5
49/49 [==============================] - 8s 160ms/step - loss: 1.5310 - accuracy: 0.4563
Epoch 3/5
49/49 [==============================] - 8s 160ms/step - loss: 1.4060 - accuracy: 0.5001
Epoch 4/5
49/49 [==============================] - 8s 160ms/step - loss: 1.3325 - accuracy: 0.5296
Epoch 5/5
49/49 [==============================] - 8s 160ms/step - loss: 1.2712 - accuracy: 0.5562
3
Epoch 1/5
49/49 [==============================] - 1s 16ms/step - loss: 1.8999 - accuracy: 0.3283
Epoch 2/5
49/49 [==============================] - 1s 16ms/step - loss: 1.5223 - accuracy: 0.4735
Epoch 3/5
49/49 [==============================] - 1s 16ms/step - loss: 1.3888 - accuracy: 0.5217
Epoch 4/5
49/49 [==============================] - 1s 16ms/step - loss: 1.3045 - accuracy: 0.5507
Epoch 5/5
49/49 [==============================] - 1s 15ms/step - loss: 1.2390 - accuracy: 0.5738
7
Epoch 1/5
49/49 [====

...

49/49 [==============================] - 12s 253ms/step - loss: 14.5933 - accuracy: 0.2645
Epoch 2/5
49/49 [==============================] - 12s 240ms/step - loss: 1.4753 - accuracy: 0.4869
Epoch 3/5
49/49 [==============================] - 12s 239ms/step - loss: 1.2011 - accuracy: 0.5831
Epoch 4/5
49/49 [==============================] - 12s 239ms/step - loss: 1.0493 - accuracy: 0.6366
Epoch 5/5
49/49 [==============================] - 12s 240ms/step - loss: 0.9155 - accuracy: 0.6883
5
Epoch 1/5
 2/49 [>.............................] - ETA: 4s - loss: 2.5987 - accuracy: 0.1338WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0310s vs `on_train_batch_end` time: 0.0861s). Check your callbacks.


49/49 [==============================] - 6s 118ms/step - loss: 1.8801 - accuracy: 0.3257
Epoch 2/5
49/49 [==============================] - 6s 118ms/step - loss: 1.4760 - accuracy: 0.4750
Epoch 3/5
49/49 [==============================] - 6s 118ms/step - loss: 1.3255 - accuracy: 0.5363
Epoch 4/5
49/49 [==============================] - 6s 118ms/step - loss: 1.2139 - accuracy: 0.5778
Epoch 5/5
49/49 [==============================] - 6s 118ms/step - loss: 1.1277 - accuracy: 0.6100
5
Epoch 1/5
49/49 [==============================] - 1s 13ms/step - loss: 1.9763 - accuracy: 0.2857
Epoch 2/5
49/49 [==============================] - 1s 12ms/step - loss: 1.6995 - accuracy: 0.3956
Epoch 3/5
49/49 [==============================] - 1s 12ms/step - loss: 1.6202 - accuracy: 0.4249
Epoch 4/5
49/49 [==============================] - 1s 13ms/step - loss: 1.5587 - accuracy: 0.4478
Epoch 5/5
49/49 [==============================] - 1s 12ms/step - loss: 1.5182 - accuracy: 0.4631
Generation: 4
Best fitn

49/49 [==============================] - 12s 240ms/step - loss: 15.7688 - accuracy: 0.2477
Epoch 2/5
49/49 [==============================] - 12s 240ms/step - loss: 1.4854 - accuracy: 0.4770
Epoch 3/5
49/49 [==============================] - 12s 240ms/step - loss: 1.2535 - accuracy: 0.5609
Epoch 4/5
49/49 [==============================] - 12s 240ms/step - loss: 1.1236 - accuracy: 0.6092
Epoch 5/5
49/49 [==============================] - 12s 239ms/step - loss: 1.0118 - accuracy: 0.6512
0
Epoch 1/5
 2/49 [>.............................] - ETA: 4s - loss: 25.1501 - accuracy: 0.1084WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0527s vs `on_train_batch_end` time: 0.1388s). Check your callbacks.


49/49 [==============================] - 9s 188ms/step - loss: 14.6942 - accuracy: 0.2082
Epoch 2/5
49/49 [==============================] - 9s 191ms/step - loss: 1.6969 - accuracy: 0.4168
Epoch 3/5
49/49 [==============================] - 9s 190ms/step - loss: 1.4554 - accuracy: 0.4962
Epoch 4/5
49/49 [==============================] - 9s 190ms/step - loss: 1.3109 - accuracy: 0.5459
Epoch 5/5
49/49 [==============================] - 9s 191ms/step - loss: 1.1848 - accuracy: 0.5934
4
Epoch 1/5
 2/49 [>.............................] - ETA: 6s - loss: 4.0452 - accuracy: 0.0986WARNING:tensorflow:Callbacks method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0373s vs `on_train_batch_end` time: 0.1213s). Check your callbacks.


49/49 [==============================] - 8s 159ms/step - loss: 2.0759 - accuracy: 0.2878
Epoch 2/5
49/49 [==============================] - 8s 159ms/step - loss: 1.5493 - accuracy: 0.4531
Epoch 3/5
49/49 [==============================] - 8s 159ms/step - loss: 1.4230 - accuracy: 0.4988
Epoch 4/5
49/49 [==============================] - 8s 159ms/step - loss: 1.3584 - accuracy: 0.5227
Epoch 5/5
49/49 [==============================] - 8s 159ms/step - loss: 1.2965 - accuracy: 0.5461
Generation: 5
Best fitness: 0.62
Average fitness: 0.52
{'a_filter_size': (3, 3), 'a_include_BN': False, 'a_output_channels': 8, 'activation_type': <class 'tensorflow.python.keras.layers.advanced_activations.ReLU'>, 'b_filter_size': (3, 3), 'b_include_BN': False, 'b_output_channels': 512, 'include_pool': False, 'pool_type': <class 'tensorflow.python.keras.layers.pooling.AveragePooling2D'>}
